# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**10/06/2020 - C11 S7**

# Ingeniería de Características y Algoritmos de Aprendizaje Automático

La práctica de la ciencia de datos se fundamenta en la obtención, exploración, limpieza, tratamiento y transformación de la información contenida en distintas fuentes de datos. Tales procedimientos constituyen gran parte del desarrollo de proyectos basados datos y su finalidad es la facilitar la obtención de conocimiento. Este último paso se lleva a cabo por medio de sistemas de aprendizaje automático. En general esto se puede resumir en las siguientes etapas:

1. **Obtención de la información**: Corresponde a la recolección de datos referentes al fenómeno de interés. Tal información puede ser tanto estructurada como no estructurada.

2. **Preprocesamiento y exploración**: Una vez reunida una porción de información significativa, se procede a explorarla y procesarla. En este apartado se aplican métodos de reducción de dimensionalidad, escalamiento, muestreo (conjuntos de entrenamiento y test), selección de características y transformación. 

3. **Aprendizaje**: Cuando los datos han sido estudiados y transformados de buena manera, se procede a la aplicación de modelos de aprendizaje de máquinas. Acá se estudian métricas de rendimiento y se optimizan hiperparámetros. 

4. **Evaluación**: Seleccionado un modelo de optimo, se generan esquemas de evaluación sobre conjuntos de datos no observados pero representativos del fenómeno estudiado, esto permite adquirir nociones sobre el comportamiento del sistema modelado fuera del conjunto de entrenamiento. 

5. **Producción**: Finalmente se genera un entorno en cual el modelo obtenido es utilizado para la obtención de información, elaboración de predicciones, clasificaciones o perfilamientos en función de la materia estudiada. 

Estos procesos son interdependientes y no presentan un orden lineal, más bien, se entrelazan circulando de manera natural entre los pasos 2 y 4. También se relacionan los pasos 1 y 6 a medida que llegan nuevos datos al sistema. 

Hasta el momento nos hemos centrado en las etapas de obtención, preprocesamiento y exploración de datos. El paso siguiente corresponde a un *refinamiento* de la información, con el fin de entrenar modelos de aprendizaje de alto rendimiento sobre los datos que se poseen. Este refinamiento ve su primera etapa en los procesos exploratorios (perfilamiento de datos y preprocesamiento inicial), para luego generar transformaciones y selecciones sobre las variables disponible. Dichas transformaciones y selecciones logran un mejor rendimiento en la medida que se obtienen por medio de *conocimiento del área* (*domain kwnlegde*), es decir, utilizando el conocimiento existente, con respecto al problema que se desea modelar. Sin embargo, existen ciertas técnicas *estándar* y algunas *automáticas* que permiten depurar los datos para obtener atributos y transformaciones de interés. 

En general, el proceso de refinamiento de datos se conoce como **ingeniería de características**, donde las características o *features* hacen referencia a los atributos y sus transformaciones. El proceso de ingeniería de características pasa generalmente por:

Explorar los datos y ajustar las variables disponibles, de manera tal, que se gane compatibilidad con las hipótesis de ciertos modelos de aprendizaje automático.

Posteriormente, se derivan características de interés, ya sea por medio de composición de atributos o transformaciones sobre estos. En esta parte, es una buena práctica, buscar transformaciones que hagan sentido con el problema modelado, esto no siempre se cumple (ej: una transformación polinomial sobre los datos no necesariamente es interpretable en ciertos problemas, pero puede aumentar considerablemente el rendimiento). 

Se continua, seleccionando caraterísticas, esto se puede hacer por medio de análisis estadísticos y algoritmos puntuación. Es necesario entender, que aunque en primera instancia más variables deberían llevarnos a mejores resultados, en la realidad, pueden aparecer caraterísticas que añadan ruido al sistema, ya sea por no estar relacionadas con el fenómeno estudiado o por presentarse problemas en su recolección. En tal contexto, cobra importancia la exlusión de dichas variables o de manera reciproca, la selección de aquellas que aseguren el mejor desempeño. 

Finalmente, se procede a entrenar y evaluar modelos sobre las métricas relevantes con el problema a trabajar, esta etapa se comunica constantemente con la de generación y selección de caraterísticas, pues las transformaciones que se decidan utilizar dependen en cierta medida de los modelos que se implementarán. 

A continuación se estudian ciertas técnicas de ingeniería de características estándar.

## Transformaciones de Atributos

Ya hemos estudiado técnicas de transformación por medio de preprocesamiento. A continuación se añaden y estudian nuevas metodologias además de aplicar algunas técnicas previas en la base de datos `HousePricing`. 

### Preprocesamiento inicial

Se cargan los datos y se agrupan sus columnas según tipo de dato

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('../S5/data/train.csv', index_col = 'Id')

Una de las técnicas más básicas de ingeniería de características es la **modificación de tipo de dato**, en este caso se transforman los datos reconocido como tipo `object` a tipo `str`.

In [ ]:
object_type_set = [col for col in df.columns if df[col].dtype == 'O']
df = df.astype({col:'str' for col in object_type_set})

Para trabajar de manera más sencilla con los datos se procede a agrupar los atributos por tipo de dato

In [ ]:
cat_cols = [
    'MSZoning', 'Street', 'Alley', 'LandContour', 'LotConfig', 'Neighborhood',
    'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
    'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation',
    'Heating', 'CentralAir', 'Electrical', 'GarageType', 'PavedDrive',
    'MiscFeature', 'SaleType', 'SaleCondition'
]

ordinal_cols = [
    'LotShape', 'Utilities', 'LandSlope', 'ExterQual', 'ExterCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC',
    'KitchenQual', 'Functional', 'FireplaceQu', 'GarageFinish', 'GarageQual',
    'GarageCond', 'PoolQC', 'Fence'
]


num_cols = [
    'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
    'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
    'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
    'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
    'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
    'MoSold', 'YrSold','SalePrice'
]

Se genera un *mapping* para obtener un datset multindexado

In [ ]:
mapping = [('numeric', col) for col in num_cols]
mapping.extend([('categorical', col) for col in cat_cols])
mapping.extend([('ordinal', col) for col in ordinal_cols])

df = df.reindex(columns=num_cols + cat_cols + ordinal_cols)
df.columns = pd.MultiIndex.from_tuples(mapping)

Obtenemos visualizaciones según tipo de variable

In [ ]:
def type_plotter(df, dtype, nrows=6):
    '''Permite graficar subconjuntos de variables en un dataframe multindexado
    
    Toma como argumento un dataframe de pandas df multi-indexado y un nivel 
    de multi - indice asociado a un subconjunto de columnas. Obtiene graficos
    univariados asociados a dicho subconjunto de varaibles.
    
    Args:
    -----
    
    df: DataFrame
        El conjunto de datos a explorar.
    
    dtype: String
        El nombre del nivel a visualizar
    
    nrwos: int
        La cantidad de filas a generar en la matriz de graficos
    
    Returns: 
    -------        
        None
        Se genera una visualizacion de matplotlib.
    '''

    cols = df[dtype].shape[1] // nrows
    resto = df[dtype].shape[1] % nrows

    fig, ax = plt.subplots(nrows=nrows + 1, ncols=cols, figsize=[17, 17])

    # Se remueve el resto de plots
    list(map(lambda a: a.remove(), ax[-1, -(cols - resto):]))

    fig.tight_layout()
    # Se define un titulo y su ubicacion
    fig.suptitle('Distribuciones Univariadas typo: ' + dtype,
                 fontsize=20,
                 x=0.5,
                 y=1.05)
    '''
    Se recorre cada axis, para cada columna del dataframe, se genera un grafico 
    distinto en funcion del tipo de dato.

    '''
    df_cols = df[dtype].columns
    for axis, col in zip(ax.flatten(), df_cols):
        try:
            # Graficos para datos numericos
            sns.distplot(df[(dtype, col)], ax=axis, rug=True)

        except RuntimeError:
            sns.distplot(df[(dtype, col)], ax=axis, rug=True, kde=False)

        except ValueError:
            sns.countplot(df[(dtype, col)], ax=axis)

        axis.set_xlabel(col, fontsize=15)

    # Se ajusta el espaciado interno entre subplots
    w, h = (.4, .4)
    plt.subplots_adjust(wspace=w, hspace=h)

Generamos visualizaciones para las variables numéricas

In [ ]:
type_plotter(df,'numeric')

El manejo de valores faltantes corresponde a una técnica de preprocesamiento ya estudiada y previa a las transformaciones de atributos buscadas en ingeniería de características. Por tal motivo, se hace un tratamiento rápido de los valores faltantes, sin ahondar en detalles. En este apartado, se reemplazan las variables con texto 'nan' por su objeto equivalente

In [ ]:
df.replace('nan',np.nan, inplace = True) 

Se observa la distribución porcentual de los valores perdidos

In [ ]:
def find_missing_per(df):
    na_percent = (df.isnull().sum()/len(df))[(df.isnull().sum()/len(df))>0].sort_values(ascending=False)

    return pd.DataFrame({'Missing Percentage':na_percent*100})

find_missing_per(df)

Se genera una función auxiliar para tratar con multi indices

In [ ]:
def indexer(cols, t_c = df.columns):
    '''Genera columnas multinivel a partir de nombres de columna planos.'''
    
    set_to_tuple = set(*[cols])

    tuples = [
        i for i in t_c if set_to_tuple.intersection(set(i))
    ]
    
    return tuples

Se llenan los valores faltantes

In [ ]:
# Se transforma nan -> None
to_none = [
    'FireplaceQu', 'Fence', 'Alley', 'MiscFeature', 'PoolQC', 'MSSubClass',
    'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtQual',
    'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType'
]

# Fill mapper
to_fill = {key: 'None' for key in indexer(to_none)}

# Se transforma nan -> 0
to_0 = [
    'GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 'BsmtFinSF2',
    'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'MasVnrArea'
]
to_fill.update({key: 0 for key in indexer(to_0)})

# Se llenan los valores faltantes
df.fillna(to_fill, inplace=True)

Se estudia el dataset luego de llenar los valores anteriores

In [ ]:
find_missing_per(df)

En este caso se tiene que las variables `LotFrontage` y `Electrical` presentan valores faltantes. Para la variable `Electrical` esto representa una cantidad muy baja de observaciones, por lo que simplemente se pueden eliminar, mientras que para la variable `LotFrontage` se escoge llenar los valores faltantes por medio de una subagrupación en función de la variable 'Neighborhood', con esto, se asigna el valor de la mediana de 'LotFrontage' en para cada subgrupo. 

In [ ]:
group_median = lambda x: x.fillna(x.median())

df.dropna(axis=0, subset=indexer(['Electrical']), inplace= True)

df[indexer(['LotFrontage'])] = df.groupby(indexer(['Neighborhood']))[indexer(['LotFrontage'])].transform(
    group_median)

finalmente no se tienen valores faltantes

In [ ]:
len(find_missing_per(df))

El siguiente paso es trabajar con los valores anómalos. Nuevamente, este paso es previo a la transformación de atributos por lo que no se estudia en profundidad.

In [ ]:
varlist = [
    'GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'MasVnrArea', 'GarageArea',
    'TotRmsAbvGrd'
]

from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN


def scatter_vs_price(df=df,
                     varlist=varlist,
                     nrows=3,
                     db_min_samples=5,
                     db_eps=0.99):
    '''Grafica una lista de variables contra SalePrice.
    
    Acepta como argumento una lista de variables, cada una de estas es 
    escalada de manera robusta, posteriormente se entrena una aglomeracion
    usando dbscan para resaltar los posible outliers.
    
    Parameters:
    -----------
    
    df: Pandas DataFrame
        El conjunto de datos de HousePricing
    
    varlist : List
              Lista de variables a estudiar
    
    nrows: int
           Cantidad de filas a graficar (argumento de subplots)
    
    db_min, db_eps: int, float
                    Hiperparametros de DBSCAN
    
    Returns:
    --------
        Visualizacion: None
                       Visualizacion de las variables estudiadas contra el precio
        
        Outliers: Dict
                  Diccionario de posibles outliers
    '''
    outliers = dict()
    
    ncols = len(varlist) // nrows
    resto = len(varlist) % nrows

    fig, ax = plt.subplots(nrows + 1, ncols, figsize=[13, 10])
    # Se borran las axis innecesarias
    for a in ax[-1, -(ncols - resto):]:
        a.remove()
    ax = ax.ravel()

    for a, var in zip(ax, varlist):
        
        X = df[indexer([var, 'SalePrice'])]
        scaler = RobustScaler()
        X = scaler.fit_transform(X)

        outlier_detection = DBSCAN(min_samples=db_min_samples, eps=db_eps)
        C = outlier_detection.fit_predict(X)

        a.scatter(x=X[:, 0], y=X[:, 1], c=C, alpha=0.5)

        #a.axvline(x=4600, color='r', linestyle='-')
        title = 'Scaled' + var + '- Price scatter plot'
        a.set_title(title, fontsize=15, weight='bold')
        outliers.update({var:C})
        
    fig.tight_layout()
    return outliers

Se estudian las variables seleccionadas

In [ ]:
outliers = scatter_vs_price()

Por simplicidad, se eliminan los valores anómalos.

In [ ]:
# Se recolectan los indices de valores anomalos
outlier_idx = pd.Index({})
for key, val in outliers.items():

    outlier_locs = [*map(bool, val)]
    columns = indexer([key])

    outlier_idx = outlier_idx.union(df.loc[outlier_locs, columns].index)
    
# Se eliminan las filas con valores anomalos
df.drop(index=outlier_idx, axis=0, inplace=True)

Si bien, luego de este preprocesamiento se puede trabajar en transformaciones sobre los datos, se debe recordar, que las etapas de preprocesamiento y manejo de caraterísticas se comunican constantemente, generando ciclos en el flujo de trabajo. 

### Manejo de Carácteristicas Básico


A continuación se procede a producir características relevantes, una primera aproximación a este procedimiento es comenzar por introducir transformaciones sobre las variables y modelar la interacción entre estas. En este apartado, es necesario discutir el término de **Data Leakage**. 

**Data Leakage** hace referencia a la *fuga* de información entre conjuntos destinados para entrenamiento y test. Si se produce esta fuga, significa que se esta utilizando información no valida en procesos de entrenamiento, lo cual genera modelos sobre optimistas en sus métricas de evaluación. En la práctica, un modelo con fuga de información puede ser incluso inútil en producción. Para evitar este fenómeno, basta realizar las transformaciones de datos pertinentes dentro de los procesos de validación y entrenamiento. Otra buena práctica en este aspecto, es generar un conjunto de validación adicional para comprobar métricas de rendimiento luego de realizar selección de modelo.

A modo de ejemplo: si en un problema de modelamiento se normalizan ciertas variables utilizando la información total del dataset y luego se procede a generar particiones de entrenamiento y validación, entonces los elementos ubicados en los conjuntos de validación habrán aportado información al proceso de normalización pues influyeron en los calculos de media y desviación estándar. Esto puede mejorar de cierta manera el rendimiento en predicción, dando como resultado un modelo sesgado.

Lo primero que se hará es generar un conjunto de validación utilizando el 10% de los datos, este conjunto se utiliza al final del proceso de selección de modelos para verificar la capacidad de generalización.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(df, test_size = .1)

A continuación, se procede a estudiar ciertos atributos del conjunto de datos

In [ ]:
f, ax = plt.subplots(figsize=(8, 7))

sns.set_style("white")
sns.set_color_codes(palette='deep')


sns.distplot(df_train['numeric']['SalePrice'], color="b");
ax.xaxis.grid(False)
ax.set(ylabel="Frecuancia")
ax.set(xlabel="SalePrice")
ax.set(title="Distribucion de SalePrice")

sns.despine(trim=True, left=True)
plt.show()

Vemos que tiene similitud a una distribución normal, para ello se hace un test de normalidad

In [ ]:
from scipy import stats

def norm_test(data = df_train['numeric']['SalePrice']):

    k, p = stats.normaltest(data)

    alpha = 1e-3
    print("p =",p)

    if p < alpha: 
        print("Se puede rechazar la hipotesis nula")
    else:
        print("No se puede rechazar la hipotesis nula")

In [ ]:
norm_test()

Luego la variable `SalePrice` no se comporta como una normal. En estos casos. Una técnica de manejo de características básica es la transformación de Box-Cox (ya vista), Podemos transformar esta variable en función de tal método

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Se utiliza una funcion auxiliar para transformar la serie
to_array = lambda pd_series: pd_series.values.reshape(-1, 1)
y = to_array(df_train['numeric']['SalePrice'])

transformer_bc = PowerTransformer(method='box-cox')

y = transformer_bc.fit_transform(y)

# Se aplica el test de normalidad a la variable de interes
norm_test(y)

Luego la transformación de Box - Cox permite normalizar de manera sencilla la distribución de la variable `SalePrice`. En general, todas las técnicas de preprocesamiento vistas anteriormente (escalar, normalizar,códificación dummy, etc...) entran en el manejo de básico de características. 

**Ejercicio**

1. Visualice las variables numéricas, calcule su asimetría estadística (skewness) por medio del método `.skew()`. Defina un valor umbral (ej: 0.6). Luego aplique una transformación de potencia sobre las variables más asimétricas. 

**Obs**: Lo anterior permite normalizar de manera más robusta aquellas variables que por naturaleza se alejan de ser normales.

Otra manera de generar características de interés es por medio de interacciones. En este caso se hace uso del conocimiento del área que se trabaja. 

Por ejemplo, las variables `TotalBsmtSF`, `1stFlrSF` y `2ndFlrSF` representan unidades de área en pies cuadrados para el subterraneo, primer y segundo piso. Con esta información se puede crear la variable `TotalSF` que representa la superficie total de una vivienda

In [ ]:
df_train[('numeric','TotalSF')] = df_train['numeric']['TotalBsmtSF'] + df_train['numeric']['1stFlrSF'] + df_train[
    'numeric']['2ndFlrSF']

Se puede también general la variable `YearsSinceRem` que permite codificar la cantidad de años que han pasado desde la remodelación de una vivienda

In [ ]:
df_train[('numeric','YearsSinceRemodel')] = df_train['numeric']['YrSold'] - df_train['numeric']['YearRemodAdd']

De la misma manera, se puede agregar la variable `TotalQual` que representa el puntaje total de calidad asociado a una vivienda

In [ ]:
df_train[('numeric','TotalQual')]  = df_train['numeric']['OverallQual'] + df_train['numeric']['OverallCond']

En general, se pueden aplicar transformaciones exponenciales (como en el caso de Box-Cox), lineales (como los dos casos anteriores) o de cualquier otro tipo (trigonometricas, polinomiales) en función del tipo de problema que se trabaje, siempre y cuando la transformación tenga un trasfondo en el fenómeno.

**Ejercicio**

Otra forma de inducir interacciones es por medio de variables dummy, en este caso se procede a generar codificaciones one hot. En este contexto, si se posee una variable codificada de esta forma y se multiplica por otra, ocurre que la nueva variable generada modela la interesección de eventos entre esas variables. 

1. Visualice las variables categóricas, genere un esquema de codificación y modele interacciones entre variables. Cuantifique estadísticamente el efecto de las variables de interacción con la variable de respuesta. 

## Selección de características 

En conjunción con el estudio, perfilamiento y aplicación de transformaciones iniciales a los datos, se hace necesario utilizar técnicas de selección de característcas. Estas permiten mejorar métricas de rendimiento en predicción. Sci-kit Learn provee de la API `sklearn.feature_selection ` que proporciona métodos de ayuda en este apartado.

A continuación se estudian métodos de **selección de caraterísticas univariadas**.

### Umbral de varianza 

Este método es un acercamiento simple y se basa en la premisa de que aquellas características con muy poca varianza no aportan información al modelo. 

**Ejemplo**

Se observan las características ordinales del conjunto de entrenamiento


In [ ]:
type_plotter(df_train,dtype='ordinal')

En este caso se observa que las variables `Functional` y `PoolQc` poseen una baja varianza (entre otras variables). Se importa el método de seleccion de caraterísticas por medio de umbral de varianza y se estudia su efecto en el conjunto de entrenamiento, para ello, es necesario codificar las variables estudiadas.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

Se generan pipelines sobre las variables ordinales

In [ ]:
# Adquieren las categorias de cada variable
ordinal_cat = [['Reg', 'IR1', 'IR2', 'IR3'],
               ['AllPub', 'NoSewr', 'NoSeWa', 'ELO'], ['Gtl', 'Mod', 'Sev'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'No', 'Mn', 'Av', 'Gd'],
               ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'],
               ['Po', 'Fa', 'TA', 'Gd', 'Ex'], ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'],
               ['None', 'Po', 'Fa', 'TA', 'Gd',
                'Ex'], ['None', 'Unf', 'RFn', 'Fin'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'Fa', 'TA', 'Gd', 'Ex'],
               ['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']]

ord_pipe = Pipeline(
    steps=[('ordinal', OrdinalEncoder(categories = ordinal_cat))])

In [ ]:
X_train = df_train.drop(('numeric','SalePrice'), axis=1).dropna(how = 'all').copy()

# Variable dependiente
y_train = df_train['numeric']['SalePrice'].copy()

# Se preparan los datos
X_prep = ord_pipe.fit_transform(X_train['ordinal'])

In [ ]:
X_train_enc = pd.DataFrame(X_prep, columns=X_train['ordinal'].columns).astype('int')

Las varianzas asociadas al conjunto de características anterior viene dado por:

In [ ]:
X_train_enc.var()

Se importa el método de umbral de varianza y se aplica a las características del conjunto de datos

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=.47)
X_filtered = sel.fit_transform(X_train_enc)

In [ ]:
X_train_enc.shape

In [ ]:
 * 1 * np.array(X_train['ordinal'].columns)

In [ ]:
X_filtered.shape

Con lo que se filtran 9 caracteristicas, para obtener las caraterísticas seleccionadas se hace uso del método `.get_support()` para obtener una mascara de los indices seleccionados

In [ ]:
X_train['ordinal'].columns[sel.get_support()]

### Métodos de Scoring
### Seleccion según percentil
### RFE